# Prompting

1. Familiarise yourself with prompting strategies such as those mentioned by [Dair ai](https://github.com/dair-ai/Prompt-Engineering-Guide)

In [11]:
import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from dotenv import load_dotenv
import os
load_dotenv() 

True

In [16]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [13]:
HF_TOKEN = os.getenv("HF_TOKEN")
print(HF_TOKEN)

In [14]:
login(token=HF_TOKEN)  

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [17]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [18]:

def print_example(prompt, temperature=0.7, top_p=0.95, repetition_penalty=1.0):
    generate_kwargs = dict(
        temperature=temperature,
        max_new_tokens=256,
        top_p=top_p,
        repetition_penalty=repetition_penalty
    )

    print("Prompt: ", prompt)

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate the output
    outputs = model.generate(**inputs, **generate_kwargs)

    # Decode and print the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_text)

In [ ]:
# TODO: Play with these parameters
 Parameters for generation
temperature = 0.7
top_p = 0.95
repetition_penalty = 1.0


 Enter any new lines as \n
 TODO: Engineer this prompt so that the model answers as many examples correctly as possible
prompt = """YOUR NICE Prompt"""

 Example problem
example = """Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?"""
prompt += example

# Call the function
print_example(prompt, temperature, top_p, repetition_penalty)

# Todo
1. Take top 10 examples of the[gsm8k](https://huggingface.co/datasets/openai/gsm8k?row=4)e dataset and compute the answers with the LLM and check against the ground truth.
2. Calculate the accuracy and report.

In [20]:
from datasets import load_dataset
dataset = load_dataset("gsm8k", "main") 
dataset

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})

In [ ]:
def evaluate(testset, config, n = 10, show_example = True, verbose = False,):
  # It takes about 13s per example, 2min for 10 examples, 20min for 100 examples
  test = testset.select(range(n))
  num_correct = 0
  for idx, example in tqdm(enumerate(test), total=len(test)):
    target = example["answer"].split("####")[1].strip()
    prompt = f"{config['prompt']}\n{example['question']}\n"
    output = # ToDo: add the function to generate the value
    prediction = output.split("Answer:")[1].strip() if 'Answer:' in output else False
    correct = target == prediction
    num_correct += correct
    if show_example: #and idx == 0: #random.randint(0, len(test)):
      if verbose:
        print(f"Showing random example: {idx}\n\nPrompt:\n{prompt}\n\nAnswer:\n{example['answer']}\n\nOutput:\n{output}")
      print(f"\nPrediction: {prediction}\nTarget: {target}\nCorrect: {correct}")
  return num_correct / n